In [23]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, classification_report
import joblib

RANDOM_STATE = 42

df = pd.read_csv('psychological_state_dataset.csv')
print(f"Данные загружены: {df.shape[0]} строк, {df.shape[1]} столбцов")

features = [
    'HRV (ms)', 'GSR (μS)', 'EEG Power Bands', 'Blood Pressure (mmHg)',
    'Oxygen Saturation (%)', 'Heart Rate (BPM)', 'Ambient Noise (dB)',
    'Cognitive Load', 'Respiration Rate (BPM)', 'Skin Temp (°C)',
    'Focus Duration (s)', 'Task Type', 'Age', 'Gender', 'Educational Level', 'Study Major'
]

target = 'Mood State'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in numeric_cols]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, cat_cols)
])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': [3, 5, 8, 12, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

f1_final = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1: {f1_final:.4f}")
print(classification_report(y_test, y_pred))

joblib.dump(best_model, 'best_model.joblib')
print("Модель сохранена в 'best_model.joblib'")

if f1_final > 0.23:
    print('Условие зачисления выполнено ✅')
else:
    print('Условие зачисления не выполнено ❌')


Данные загружены: 1000 строк, 20 столбцов
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Лучшие параметры: {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 5}
Weighted F1: 0.2426
              precision    recall  f1-score   support

     Anxious       0.16      0.17      0.16        48
       Happy       0.30      0.36      0.32        53
     Neutral       0.27      0.25      0.26        48
         Sad       0.24      0.20      0.22        51

    accuracy                           0.24       200
   macro avg       0.24      0.24      0.24       200
weighted avg       0.24      0.24      0.24       200

Модель сохранена в 'best_model.joblib'
Условие зачисления выполнено ✅
